# Imports and inits

In [32]:
import numpy as np
import sympy as sp
from sympy import cos, sin

In [40]:
t = sp.Symbol('t')
qs = [sp.Function(f'q_{i}')(t) for i in range(6)] # Joints effect
ls = sp.symbols(r"l_:6") # Link lengths
hs = sp.symbols(r"h_:3") # Link heights
ws = sp.symbols(r"w_:3") # Link widths
rs = sp.symbols(r"r_:6") # Link radii

In [41]:
qs

[q_0(t), q_1(t), q_2(t), q_3(t), q_4(t), q_5(t)]

In [42]:
# Source: symbolic_fk.ipynb demo code

def rz(theta):
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,:] = [[cos(theta), -sin(theta),0,0]]
    rot[1,:] = [[sin(theta), cos(theta),0,0]]
    rot[2,2] = 1

    return rot

def rx(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,0] = 1
    rot[1,:] = [[0, cos(theta), -sin(theta),0]]
    rot[2,:] = [[0, sin(theta), cos(theta),0]]

    return rot


def ry(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[1,1] = 1
    rot[0,:] = [[ cos(theta), 0, sin(theta),0]]
    rot[2,:] = [[-sin(theta),0, cos(theta),0]]

    return rot


def trans(vector):
    mat = sp.eye(4)
    mat[0:3,3] = vector
    return mat

def tx(dx):
    return trans([dx, 0, 0])

def ty(dy):
    return trans([0, dy, 0])

def tz(dz):
    return trans([0, 0, dz])

In [57]:
# Recursive forward kinematics
def forward(i, j):
    assert i <= j
    assert j < 6

    T = [[None] * 6] * 6
    T[0][1] = tz(qs[1])
    T[1][2] = tz(hs[1] / 2 + ws[2] / 2) @ tx(qs[2])
    T[2][3] = tx(hs[2] / 2 + rs[3]) @ rz(qs[3])
    T[3][4] = tz(ls[3] / 2 + rs[4]) @ ry(qs[4])
    T[4][5] = tx(rs[4] + ls[5] / 2) @ rx(qs[5])

    if i == j:
        return sp.eye(4)
    if i - 1 == j:
        return T[i][j]
    
    return forward(i, j - 1) @ T[j - 1][j]

# Transformations

In [62]:
T = forward(0, 5)
T.simplify()
print('Complete transformation matrix:')
display(T)
print('Translation:')
display(T[0:3,3])
print('Rotation:')
display(T[0:3,0:3])

Complete transformation matrix:


Matrix([
[cos(q_3(t))*cos(q_4(t)), -sin(q_3(t))*cos(q_5(t)) + sin(q_4(t))*sin(q_5(t))*cos(q_3(t)), sin(q_3(t))*sin(q_5(t)) + sin(q_4(t))*cos(q_3(t))*cos(q_5(t)),       h_2/2 + r_3 + (l_5/2 + r_4)*cos(q_3(t))*cos(q_4(t)) + q_2(t)],
[sin(q_3(t))*cos(q_4(t)),  sin(q_3(t))*sin(q_4(t))*sin(q_5(t)) + cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t)),                              (l_5/2 + r_4)*sin(q_3(t))*cos(q_4(t))],
[           -sin(q_4(t)),                                        sin(q_5(t))*cos(q_4(t)),                                       cos(q_4(t))*cos(q_5(t)), h_1/2 + l_3/2 + r_4 + w_2/2 - (l_5 + 2*r_4)*sin(q_4(t))/2 + q_1(t)],
[                      0,                                                              0,                                                             0,                                                                  1]])

Translation:


Matrix([
[      h_2/2 + r_3 + (l_5/2 + r_4)*cos(q_3(t))*cos(q_4(t)) + q_2(t)],
[                             (l_5/2 + r_4)*sin(q_3(t))*cos(q_4(t))],
[h_1/2 + l_3/2 + r_4 + w_2/2 - (l_5 + 2*r_4)*sin(q_4(t))/2 + q_1(t)]])

Rotation:


Matrix([
[cos(q_3(t))*cos(q_4(t)), -sin(q_3(t))*cos(q_5(t)) + sin(q_4(t))*sin(q_5(t))*cos(q_3(t)), sin(q_3(t))*sin(q_5(t)) + sin(q_4(t))*cos(q_3(t))*cos(q_5(t))],
[sin(q_3(t))*cos(q_4(t)),  sin(q_3(t))*sin(q_4(t))*sin(q_5(t)) + cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t))],
[           -sin(q_4(t)),                                        sin(q_5(t))*cos(q_4(t)),                                       cos(q_4(t))*cos(q_5(t))]])